In [ ]:
!pip install bs4 pandas openpyxl requests

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import Workbook

# Define the base URL
BASE_URL = "https://www.advisorkhoj.com/Mumbai/Share-Trading-&-Demat-Advisor-%3Cnear%3E-Pin-Code-{pincode}?distance=40&experience=1&sortby=Recommended"

# Function to extract data from a specific pin code
def extract_data(pincode):
    url = BASE_URL.format(pincode=pincode)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    names, emails, phones = [], [], []

    # Extract advisor details if available
    advisors = soup.find_all('div', class_='panel-body')
    for advisor in advisors:
        name_tag = advisor.find('h2', class_='advisor-name')
        email_tag = advisor.find('a', class_='brand-links bold')
        phone_tag = advisor.find('a', class_='brand-links bold advisor-mobile')

        name = name_tag.get_text(strip=True) if name_tag else "N/A"
        email = email_tag.get_text(strip=True) if email_tag else "N/A"
        phone = phone_tag['title'] if phone_tag and phone_tag.has_attr('title') else "N/A"

        names.append(name)
        emails.append(email)
        phones.append(phone)

    # Return the extracted data as a DataFrame
    return pd.DataFrame({'Name': names, 'Email': emails, 'Phone': phones})

# Read pin codes from the uploaded Excel file
input_file = '/content/Pincodes Mumbai.xlsx'
pincode_data = pd.read_excel(input_file)
pincodes = pincode_data['Pincode'].astype(str)  # Ensure pincode is a string

# Create an Excel writer to save results
output_file = '/content/Details_from_Region.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for pincode in pincodes:
        print(f"Processing pincode: {pincode}")
        try:
            data = extract_data(pincode)
            if not data.empty:
                # Add "Sl. No." column starting from 1
                data.insert(0, 'Sl. No.', range(1, len(data) + 1))

                # Write to Excel sheet for this pincode
                data.to_excel(writer, sheet_name=f"Pincode_{pincode}", index=False)
        except Exception as e:
            print(f"Error processing pincode {pincode}: {e}")

print(f"Data extraction complete. Consolidated data saved to {output_file}.")
